# নামযুক্ত সত্তা সনাক্তকরণ (NER)

এই নোটবুকটি [AI for Beginners Curriculum](http://aka.ms/ai-beginners) থেকে নেওয়া হয়েছে।

এই উদাহরণে, আমরা শিখব কীভাবে [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) ডেটাসেট ব্যবহার করে NER মডেল প্রশিক্ষণ দিতে হয়। শুরু করার আগে, অনুগ্রহ করে [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ফাইলটি বর্তমান ডিরেক্টরিতে ডাউনলোড করুন।


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## ডেটাসেট প্রস্তুত করা

আমরা ডেটাসেটটি একটি ডেটাফ্রেমে পড়ে শুরু করব। যদি আপনি Pandas ব্যবহার সম্পর্কে আরও জানতে চান, তবে আমাদের [ডেটা প্রসেসিং পাঠ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) এবং [ডেটা সায়েন্স ফর বিগিনারস](http://aka.ms/datascience-beginners) দেখুন।


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


চলুন অনন্য ট্যাগগুলি পাই এবং লুকআপ ডিকশনারি তৈরি করি যা আমরা ট্যাগগুলিকে ক্লাস নম্বরে রূপান্তর করতে ব্যবহার করতে পারি:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

এখন আমাদের শব্দভাণ্ডারের সাথে একই কাজ করতে হবে। সহজতার জন্য, আমরা শব্দের ফ্রিকোয়েন্সি বিবেচনা না করে শব্দভাণ্ডার তৈরি করব; বাস্তব জীবনে আপনি কেরাস ভেক্টরাইজার ব্যবহার করতে পারেন এবং শব্দের সংখ্যা সীমিত করতে পারেন।


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

আমাদের প্রশিক্ষণের জন্য বাক্যগুলির একটি ডেটাসেট তৈরি করতে হবে। চলুন মূল ডেটাসেটের মাধ্যমে লুপ করি এবং সমস্ত পৃথক বাক্যগুলোকে `X` (শব্দের তালিকা) এবং `Y` (টোকেনের তালিকা) এ বিভক্ত করি:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

সাধারণতার জন্য, আমরা সমস্ত বাক্যকে সর্বাধিক দৈর্ঘ্যে 0 টোকেন দিয়ে পূরণ করব। বাস্তব জীবনে, আমরা আরও বুদ্ধিমান কৌশল ব্যবহার করতে পারি এবং শুধুমাত্র একটি মিনিব্যাচের মধ্যে ক্রমগুলো পূরণ করতে পারি।


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## টোকেন শ্রেণীবিন্যাস নেটওয়ার্ক সংজ্ঞায়িত করা

আমরা টোকেন শ্রেণীবিন্যাসের জন্য দুই-স্তর বিশিষ্ট দ্বিমুখী LSTM নেটওয়ার্ক ব্যবহার করব। শেষ LSTM স্তরের প্রতিটি আউটপুটে ঘন শ্রেণীবিন্যাসকারী প্রয়োগ করার জন্য, আমরা `TimeDistributed` নির্মাণ ব্যবহার করব, যা প্রতিটি ধাপে LSTM-এর প্রতিটি আউটপুটে একই ঘন স্তর পুনরাবৃত্তি করে:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

আমাদের ডেটাসেটের জন্য এখানে আমরা স্পষ্টভাবে `maxlen` নির্ধারণ করছি - যদি আমরা চাই নেটওয়ার্কটি পরিবর্তনশীল দৈর্ঘ্যের সিকোয়েন্স পরিচালনা করতে সক্ষম হয়, তাহলে নেটওয়ার্ক সংজ্ঞায়িত করার সময় আমাদের আরও বুদ্ধিমত্তার সাথে কাজ করতে হবে।

এখন আমরা মডেলটি প্রশিক্ষণ করব। গতি বাড়ানোর জন্য, আমরা শুধুমাত্র এক ইপোকের জন্য প্রশিক্ষণ করব, তবে আপনি চাইলে দীর্ঘ সময় ধরে প্রশিক্ষণ করতে পারেন। এছাড়াও, আপনি ডেটাসেটের কিছু অংশকে প্রশিক্ষণ ডেটাসেট হিসেবে আলাদা করতে পারেন, যাতে যাচাইয়ের সঠিকতা পর্যবেক্ষণ করা যায়।


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## ফলাফল পরীক্ষা করা

এবার চলুন দেখি আমাদের সত্তা স্বীকৃতি মডেল একটি নমুনা বাক্যে কীভাবে কাজ করে:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## মূল বিষয়বস্তু

এমনকি একটি সাধারণ LSTM মডেলও NER-এ যুক্তিসঙ্গত ফলাফল দেখায়। তবে, আরও ভালো ফলাফল পেতে, আপনি BERT-এর মতো বড় প্রি-ট্রেইনড ভাষার মডেল ব্যবহার করতে পারেন। Huggingface Transformers লাইব্রেরি ব্যবহার করে NER-এর জন্য BERT ট্রেইনিং করার প্রক্রিয়া [এখানে](https://huggingface.co/course/chapter7/2?fw=pt) বর্ণনা করা হয়েছে।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
